In [1]:
def grafico_pontos(*args, return_figure = False):
    import types
    from bokeh.plotting import figure, show
    from bokeh.io import output_notebook
    from bokeh.models import Range1d
    output_notebook()
    lista_cores = ['black', 'navy', 'red', 'green', 'orange', 'violet']
    p = figure(width=400, height=400,
               tools="hover,pan,box_zoom,wheel_zoom,zoom_in,zoom_out,reset,save")
    
    def set_range_limits(range_limits):
        if range_limits['lmin_x'] < range_limits['min_x']:
            range_limits['min_x'] = range_limits['lmin_x']
        if range_limits['lmax_x'] > range_limits['max_x']:
            range_limits['max_x'] = range_limits['lmax_x']
        if range_limits['lmin_y'] < range_limits['min_y']:
            range_limits['min_y'] = range_limits['lmin_y']
        if range_limits['lmax_y'] > range_limits['max_y']:
            range_limits['max_y'] = range_limits['lmax_y']
        return range_limits
    
    range_limits = {}
    
    if len(args)%3 == 0 and type(args[2]) == types.FunctionType:
        counter = 0
        counter_cores = 0
        for i in range(len(args)):
            counter += 1
            if counter%3 == 0:
                X = args[i - 2]
                Y = args[i - 1]
                p.circle(X, Y, size=5, color=lista_cores[counter_cores], alpha=0.5)
                counter_cores += 1
                func = args[i]
                func_vals = [func(x) for x in X]
                
                range_limits['lmin_x'], range_limits['lmax_x'] = min(X), max(X)
                range_limits['lmin_y'], range_limits['lmax_y'] = min(Y), max(Y)
                
                p.circle(X, func_vals, size=3, color="red", alpha=1)
                
                #definindo limites gráfico
                if counter == 3:
                    range_limits['min_x'], range_limits['max_x'], range_limits['min_y'], range_limits['max_y'] = range_limits['lmin_x'], range_limits['lmax_x'], range_limits['lmin_y'], range_limits['lmax_y']
                else:
                    range_limits = set_range_limits(range_limits)
                
                p.x_range = Range1d(range_limits['min_x'] - (0.2*range_limits['max_x']), range_limits['max_x'] + (0.2*range_limits['max_x']))
                p.y_range = Range1d(range_limits['min_y'] - (0.2*range_limits['max_y']), range_limits['max_y'] + (0.2*range_limits['max_y']))
    
    elif len(args)%2 == 0:
        counter_cores = 0
        for i in range(len(args)):
            if i%2 == 0:
                X = args[i]
                Y = args[i + 1]
                
                range_limits['lmin_x'], range_limits['lmax_x'] = min(X), max(X)
                range_limits['lmin_y'], range_limits['lmax_y'] = min(Y), max(Y)
                
                p.circle(X, Y, size=5, color=lista_cores[counter_cores], alpha=0.5)
                
                #definindo limites gráfico
                if i == 0:
                    range_limits['min_x'], range_limits['max_x'], range_limits['min_y'], range_limits['max_y'] = range_limits['lmin_x'], range_limits['lmax_x'], range_limits['lmin_y'], range_limits['lmax_y']
                else:
                    range_limits = set_range_limits(range_limits)
                
                p.x_range = Range1d(range_limits['min_x'] - (0.2*range_limits['max_x']), range_limits['max_x'] + (0.2*range_limits['max_x']))
                p.y_range = Range1d(range_limits['min_y'] - (0.2*range_limits['max_y']), range_limits['max_y'] + (0.2*range_limits['max_y']))
                
                counter_cores += 1
    
    else:
        print('\nERRO!')
        print('Insira como argumento da função os valores de X, Y e, caso houver, uma função algébrica.')
        print('EX 1: "grafico_pontos(x_vals, y_vals)"')
        print('EX 2: "grafico_pontos(x_vals, y_vals, função_reta)"')
        print('EX 3: "grafico_pontos(x_vals1, y_vals1, função_reta, x_vals2, y_vals2, funao_gaussiana)"')
        return
    
    if return_figure is True:
        return p
    else:
        show(p)

In [2]:
def importar_txt(caminho):
    import re
    
    #abrindo o arquivo txt
    r=open(caminho, 'r')
    linhas=r.readlines()
    
    #dic para armazenar os pontos
    points={}
    points['x'] , points['y'] = [], []
    
    #varrendo as linhas
    for i in linhas:
        vals = {}
        vals['x'] = re.findall(r'-?[0-9]+\.?[0-9]*e?\-?[0-9]*(?=\,)',i)[0]
        vals['y'] = re.findall(r'(?<=\,)-?[0-9]+\.?[0-9]*e?\-?[0-9]*',i)[0]

        #procurando por fatores de ordem de magnitude
        for j in ('x', 'y'):
            #print('text val: ', vals[j])
            base_val = re.findall(r'-?[0-9]+\.?[0-9]*', vals[j])
            #print('base val', base_val)            
            factor = re.findall(r'(?<=[0-9]e)\-?[0-9]+', vals[j])
            #print('factor', factor)

            if len(factor) > 0:
                #caso seja negativo
                if factor[0][0] == '-':
                    raw_val = float(base_val[0]) / (10**int(factor[0][1:]))
                #caso seja positivo
                else:
                    print(float(base_val[0]))
                    raw_val = float(base_val[0]) * (10**int(factor[0]))
            else:
                raw_val = float(base_val[0])
        
            rounded_val = round(raw_val, 10)
            #print(f'{j} val to add to the list: ', rounded_val)
            points[j].append(rounded_val)
    
    return points['x'], points['y']

In [3]:
def encontrar_modelo(x_vals, y_vals, func):
    from scipy.optimize import curve_fit
    popt, pcov = curve_fit(func, x_vals, y_vals)
    if len(popt) > 1:
        return list(popt)
    else:
        return popt[0]

In [4]:
import numpy as np
pi = np.pi
e = np.e
def seno(x):
    return np.sin(x)

def cosseno(x):
    return np.cos(x)

def tangente(x):
    return np.tan(x)

def exp(x):
    return np.exp(x)

<img src="logo1.png" style="width:450px;height:150px"/>

# Ilum Escola de Ciência
### Curso: Termodinâmica
### Prof: Amauri Jardim de Paula
### <font color='red'>Observação: resolução dos exercícios + autoavaliação no fim do arquivo</font>
###
### Aula 1. Introdução ao JupyterLab
###
##### Bem vindos ao JupyterLab!
##### O JupyterLab será o programa que usaremos para realizar as atividades do curso de Termodinâmica em sala de aula. 
##### Para cada atividade, um 'notebook' do JupyterLab (formato .pynb) será fornecido aos alunos. 
##### Nesse notebook haverá códigos com **funções** relevantes à atividade. Usaremos esse notebook para exemplificar.
##### A **função** definida como "graficos_pontos" é uma função para plotar dados em um gráfico de pontos.
##### Para usá-la, vamos usar alguns dados relevantes que estão armazenados em um arquivo de texto do tipo .txt. 
##### Para trazer esses dados para o notebook do JupyterLab, use a **função** que foi definida como "importar_txt".

In [5]:
x_vals, y_vals = importar_txt("Aula 1. Exemplo de Dados 1.txt")

##### A **função** "importar_txt" carrega no JupyterLab o arquivo .txt que está armazenado no seu computador.
##### O caminho do arquivo no Windows é fornecido (entre aspas ("")) como 'input' da função.
##### Após aplicada a **função** 'importar_txt' ela devolve duas listas, a primeira com os valores de 'x' e a segunda com os valores de 'y'.
##### O sinal de igual (**=**) representa uma **operação de atribuição**, ou seja, os 'outputs' da **função** 'importar_txt' (são dois) serão atribuidos aos dois termos: 'x_vals' e 'y_vals'.
##### Perceba o que acontece quando digitamos 'x_vals':

In [6]:
x_vals

[-10.0,
 -9.8994974874,
 -9.7989949749,
 -9.6984924623,
 -9.5979899497,
 -9.4974874372,
 -9.3969849246,
 -9.2964824121,
 -9.1959798995,
 -9.0954773869,
 -8.9949748744,
 -8.8944723618,
 -8.7939698492,
 -8.6934673367,
 -8.5929648241,
 -8.4924623116,
 -8.391959799,
 -8.2914572864,
 -8.1909547739,
 -8.0904522613,
 -7.9899497487,
 -7.8894472362,
 -7.7889447236,
 -7.6884422111,
 -7.5879396985,
 -7.4874371859,
 -7.3869346734,
 -7.2864321608,
 -7.1859296482,
 -7.0854271357,
 -6.9849246231,
 -6.8844221106,
 -6.783919598,
 -6.6834170854,
 -6.5829145729,
 -6.4824120603,
 -6.3819095477,
 -6.2814070352,
 -6.1809045226,
 -6.0804020101,
 -5.9798994975,
 -5.8793969849,
 -5.7788944724,
 -5.6783919598,
 -5.5778894472,
 -5.4773869347,
 -5.3768844221,
 -5.2763819095,
 -5.175879397,
 -5.0753768844,
 -4.9748743719,
 -4.8743718593,
 -4.7738693467,
 -4.6733668342,
 -4.5728643216,
 -4.472361809,
 -4.3718592965,
 -4.2713567839,
 -4.1708542714,
 -4.0703517588,
 -3.9698492462,
 -3.8693467337,
 -3.7688442211,
 -3.

##### O mesmo acontece quando digitamos 'y_vals' dentro de uma célula de código.

##### Com os dados importados, agora podemos plotar os dados utilizando a **função** 'grafico_pontos', a qual também já está pré-definida.

In [7]:
grafico_pontos(x_vals, y_vals)

Loading BokehJS ...

##### Percebam que os pontos parecem indicar que esses dados possivelmente descrevem uma *função algébrica* de uma reta.
##### Vamos ver se isso é verdadeiro fornecendo uma possível *função algébrica* para fazermos um 'fit', ou seja, uma tentativa de encontrar um modelo matemático que descreva os dados inseridos.
##### Para isso, usaremos a **função** definida como 'encontrar_modelo'
##### Mas antes de usar a **função** 'encontrar_modelo', precisamos definir uma possível *função algébrica* que descreva os valores importados de x e y.
##### Essa *função algébrica* seria do tipo y(x) = a + b.x
##### Vamos definir a **função** de uma reta em linguagem Python. 

In [8]:
def func_reta(x, a, b):
    return a + (b * x)

##### OBS: multiplicação no Python é definida pelo símbolo \* (asterisco)
##### Portanto, vamos usar a *função algébrica* 'func_reta' dentro da função 'encontrar_modelo'

In [9]:
encontrar_modelo(x_vals, y_vals, func_reta)

[3.283387388599297, 3.7215058416344955]

##### Os valores encontrados para as constantes 'a' e 'b' da equação da reta são 3.658 e 3.669, respectivamente.
##### Percebam que como resultado da **função** 'encontrar_modelo', temos somente a determinação das constantes propostas (nesse caso, a e b).
##### Já 'x' é a variável da função.
##### Portanto, no processo de modelagem dos dados com a *função algébrica* proposta (função da reta), temos que descobrir os valores das constantes.

###
##### Continuando...
##### Para facilitar o nosso trabalho, vamos atribuir dois termos aos valores de 'output' da função 'encontrar_modelo'

In [10]:
a , b = encontrar_modelo(x_vals, y_vals, func_reta)

##### Testem agora como o interpretaor reage quando digitados os termos a e b

In [11]:
a

3.283387388599297

In [12]:
b

3.7215058416344955

##### De posse dos valores das constantes 'a' e 'b', podemos definir uma **função** em Python que represente a *função algébrica* encontrada.

In [13]:
def funcao_reta_encontrada_1(x):
    return 3.658 + (3.669 * x)

##### Teste essa **função** 'funcao_reta_encontrada' inserindo possíveis valores para 'x'
#####
##### A mesma *função algébrica* poderia ser definida da seguinte forma:

In [14]:
def funcao_reta_encontrada_2(x):
    return a + (b * x)

##### Isso pois os termos 'a' e 'b' são interpretados como os valores '3.658' e '3.669' (eles foram atribuidos anteriormente).

##### Agora vamos checar como ficou o modelo, introduzindo essa **função** 'funcao_reta_encontrada2' encontrada na **função** "gráfico pontos".
##### Além disso, vamos inserir os valores de 'x' e 'y' na **função** 'grafico_pontos'

In [15]:
grafico_pontos(x_vals, y_vals, funcao_reta_encontrada_2)

Loading BokehJS ...

#
## Problema
### Modelar os dados contidos nos arquivos:
- Aula 1. Exemplo de Dados 2.txt
- Aula 1. Exemplo de Dados 3.txt
- Aula 1. Exemplo de Dados 4.txt

## Instruções
- Usar as **funções** em Python apresentadas acima para resolver esse problema.
- Criar **funções** em Python que representem *funções algébricas* que possam modelar os dados importados.
- No notebook já estão definidos **funções** em Python para o cálculo do seno, cosseno e tangente de um valor, e também do exponencial de um valor (ver abaixo).
- O valor da constante π está atribuido ao termo 'pi', e o valor da base do logaritmo natural está associada ao termo 'e' (número de Euler)
 
## Observação
##### Comentários relevantes à atividade devem ser incorporados no notebook.
##### *No final da atividade, enviar o notebook pelo Moodle*

## Avaliação
##### De 0 a 10, qual nota você daria considerando o seu desempenho nessa atividade?
##### Quais dificuldades você teve nessa atividade?

## Resolução dos Exercícios - Pedro Zanineli

**Autoavaliação**: atribuo uma nota 10 para o meu desempenho na atividade, já que, além de ter terminado os exercícios ainda em aula, também consegui ajudar os meus outros três colegas que estavam na mesa e trouxe pontos importantes para a resolução. Na aula, tive dificuldades no exemplo da Curva de Gauss.

Realizando o mesmo processo de modelagem para o exemplo 2, 3 e 4:
- Importação dos dados do arquivo;
- Plotagem dos dados;
- Definição de que tipo de função é (afim, quadrática, etc.);
- Encontrar o modelo;
- Plotar a função no gráfico com os valores.

#### Exercício 1 - Exemplo 2

In [16]:
valores_x_2, valores_y_2 = importar_txt("Aula 1. Exemplo de Dados 2.txt")

In [17]:
grafico_pontos(valores_x_2, valores_y_2)

Loading BokehJS ...

In [18]:
def func_quad(x, a, b, c):
    return (a*x**2) + b*x + c

In [19]:
a, b, c = encontrar_modelo(valores_x_2, valores_y_2, func_quad)

In [20]:
def funcao_quad_encontrada(x):
    return (a*x**2) + b*x + c

In [21]:
grafico_pontos(valores_x_2, valores_y_2, funcao_quad_encontrada)

Loading BokehJS ...

#### Exercício 2 - Exemplo 3

In [22]:
valores_x_3, valores_y_3 = importar_txt("Aula 1. Exemplo de Dados 3.txt")

In [23]:
grafico_pontos(valores_x_3, valores_y_3)

Loading BokehJS ...

In [24]:
def gauss(x, H, A, x0, sigma):
    return H + A * exp((-(x - x0) ** 2 / (2 * sigma ** 2)))

In [25]:
H, A, x0, sigma = encontrar_modelo(valores_x_3, valores_y_3, gauss)

In [26]:
def gauss_encontrada(x):
    return H + A * e**(-(x - x0) ** 2 / (2 * sigma ** 2))

In [27]:
grafico_pontos(valores_x_3, valores_y_3, gauss_encontrada)

Loading BokehJS ...

#### Exercício 3 - Exemplo 4 

In [28]:
valores_x_4, valores_y_4 = importar_txt("Aula 1. Exemplo de Dados 4.txt")

In [29]:
grafico_pontos(valores_x_4, valores_y_4)

Loading BokehJS ...

In [30]:
def func_seno(x,a,b,c):
    return a*seno(c*x) + b

In [31]:
a, b, c = encontrar_modelo(valores_x_4, valores_y_4, func_seno)

In [32]:
def funcao_seno_encontrada(x):
    return a*seno(c*x) + b

In [33]:
grafico_pontos(valores_x_4, valores_y_4, funcao_seno_encontrada)

Loading BokehJS ...

#

In [34]:
pi

3.141592653589793

In [35]:
e

2.718281828459045

In [36]:
seno(pi/2)

1.0

In [37]:
cosseno(pi)

-1.0

In [38]:
tangente(0)

0.0

In [39]:
exp(1)

2.718281828459045